In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

In [2]:
filepath = '../data/eda_data.csv'
data = pd.read_csv(filepath)

In [3]:
X = data.drop('HRFS12M1', axis=1)
y = data['HRFS12M1']

(       HURESPLI  HUFINAL  HULANGCODE  HETENURE  HEHOUSUT  HETELHHD  HETELAVL  \
 0             2      201           0         1         1         1        -1   
 1             2      201           0         1         1         1        -1   
 2             2      201           0         1         1         1        -1   
 3             2      201           0         1         1         1        -1   
 4             1      201           0         2         1         1        -1   
 ...         ...      ...         ...       ...       ...       ...       ...   
 81817         1      201           0         1         1         1        -1   
 81818         1      201           0         1         1         1        -1   
 81819         1      201           0         1         1         1        -1   
 81820         1      201           0         1         5         1        -1   
 81821         1      201           0         1         5         1        -1   
 
        HEPHONEO  HEFAMINC

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, stratify=y)